In [ ]:
import os
import shutil
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jessicali9530/lfw-dataset

In [ ]:
!unzip -q lfw-dataset.zip
print("✅ LFW Dataset is ready!")

In [ ]:
import os
import shutil
from tqdm import tqdm
import numpy as np

print("--- Creating a larger, balanced data subset ---")
# --- Configuration ---
SOURCE_DIR = 'lfw-deepfunneled/lfw-deepfunneled/'
SUBSET_DIR = 'lfw-subset-large/' # New directory for the larger subset
NUM_CLASSES = 200 # Increased from 100
MIN_IMAGES = 15
MAX_IMAGES = 80

# --- Create a Clean Subset Directory ---
if os.path.exists(SUBSET_DIR):
    shutil.rmtree(SUBSET_DIR)
os.makedirs(SUBSET_DIR)

# --- Find and Select People ---
all_identities = [d for d in os.listdir(SOURCE_DIR) if os.path.isdir(os.path.join(SOURCE_DIR, d))]
identity_counts = {identity: len(os.listdir(os.path.join(SOURCE_DIR, identity))) for identity in all_identities}
filtered_identities = {identity: count for identity, count in identity_counts.items() if MIN_IMAGES <= count <= MAX_IMAGES}
selected_identities = list(filtered_identities.keys())[:NUM_CLASSES]

print(f"Creating subset with {len(selected_identities)} identities...")
for identity in tqdm(selected_identities):
    shutil.copytree(os.path.join(SOURCE_DIR, identity), os.path.join(SUBSET_DIR, identity))
print(f"\n✅ Large subset created at '{SUBSET_DIR}'")

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

print("\n--- Loading data and building the final model ---")
# --- Configuration ---
IMG_SIZE = (260, 260)
BATCH_SIZE = 32
# Make sure SUBSET_DIR is defined with the path to your dataset
# SUBSET_DIR = "path/to/your/lfw_subset"

# --- Data Loading ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    SUBSET_DIR, validation_split=0.2, subset="training", seed=123, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    SUBSET_DIR, validation_split=0.2, subset="validation", seed=123, image_size=IMG_SIZE, batch_size=BATCH_SIZE
)
class_names = train_dataset.class_names
print(f"Loaded {len(class_names)} classes.")

# --- Data Augmentation Layer ---
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# --- Build the Model using EfficientNetB2 ---
base_model = tf.keras.applications.EfficientNetB2(
    input_shape=(260, 260, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False # Keep the base frozen for initial training

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(260, 260, 3)),
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# --- Compile and Initial Training ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# --- ADDED: Callbacks for smarter training ---
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="face_model_best.keras",
    save_best_only=True,
    monitor="val_accuracy",
    verbose=1
)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5, # Stop if validation loss doesn't improve for 5 epochs
    restore_best_weights=True,
    verbose=1
)

print("\n--- Starting Initial Training (Phase 1) ---")
# --- CHANGED: Increased epochs and added callbacks ---
initial_epochs = 50
history = model.fit(
    train_dataset,
    epochs=initial_epochs,
    validation_data=validation_dataset,
    callbacks=[checkpoint_callback, early_stopping_callback] # Use callbacks
)

# ==========================================================
# --- ADDED: Fine-Tuning Stage (Phase 2) ---
# ==========================================================
print("\n--- Starting Fine-Tuning (Phase 2) ---")
base_model.trainable = True # Unfreeze the base model

# Re-compile the model with a very low learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # 0.00001
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Continue training for a few more epochs
fine_tune_epochs = 20
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1], # Start from where we left off
    validation_data=validation_dataset,
    callbacks=[checkpoint_callback, early_stopping_callback] # Reuse the same callbacks
)

print("\n--- Training complete. Best model saved to 'face_model_best.keras' ---")

In [ ]:
print("\n--- Starting Final Fine-Tuning ---")
base_model.trainable = True

# Unfreeze the top 1/3 of the model (approx.)
fine_tune_at = len(base_model.layers) // 3
for layer in base_model.layers[:-fine_tune_at]:
    layer.trainable = False

# --- Learning Rate Scheduler Callback ---
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # Reduce LR by a factor of 5
    patience=2, # after 2 epochs of no improvement
    verbose=1,
    min_lr=1e-7 # Don't let the LR get too small
)

# Re-compile with a low initial learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Continue training with the scheduler
fine_tune_epochs = 30
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=validation_dataset,
    callbacks=[lr_scheduler] # Add the callback here
)
print("\n✅ Final fine-tuning complete!")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

print("\n--- Running Final Predictions on Validation Images ---")
plt.figure(figsize=(10, 12))
for images, labels in validation_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        img_array = tf.expand_dims(images[i], 0)
        # Use the final_model for predictions
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        confidence = 100 * np.max(predictions[0])

        actual_name = class_names[labels[i]]
        predicted_name = class_names[predicted_class_index]

        plt.title(f"Actual: {actual_name}\nPredicted: {predicted_name}\nConf: {confidence:.2f}%")
        plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# After the model.fit() call for your LFW model is complete
print("--- Saving the trained face recognition model ---")
model.save('face_recognition_lfw_model.keras')
print("Model saved successfully!")

In [ ]:
!kaggle datasets download -d mstjebashazida/affectnet

In [ ]:
!unzip -q affectnet.zip -d affectnet_data

In [ ]:
!ls affectnet_data

In [ ]:
import pandas as pd
import tensorflow as tf
import os

# Configuration
IMG_SIZE = (260, 260)
BATCH_SIZE = 32
DATA_DIR = 'affectnet_data/'

In [ ]:
DATA_DIR = 'affectnet_data/'
IMAGE_BASE_DIR = os.path.join(DATA_DIR, 'archive (3)', 'Train/')
csv_path = os.path.join(DATA_DIR, 'archive (3)', 'labels.csv')
IMG_SIZE = (260, 260)
BATCH_SIZE = 32

In [ ]:
print("--- Loading and processing DataFrame ---")
df = pd.read_csv(csv_path)
df['full_path'] = IMAGE_BASE_DIR + df['pth']
class_names = sorted(df['label'].unique())
label_to_id = {label: i for i, label in enumerate(class_names)}
df['numeric_label'] = df['label'].map(label_to_id)
print("DataFrame successfully processed.")

In [ ]:
print("--- DataFrame Head ---")
print(df.head())
print("\n--- DataFrame Info (Check Dtypes) ---")
df.info()
print("-" * 30)

In [ ]:
print("\n--- Creating tf.data.Dataset ---")
image_paths = df['full_path'].values
emotion_labels = df['numeric_label'].values
dataset = tf.data.Dataset.from_tensor_slices((image_paths, emotion_labels))

In [ ]:
class_names = sorted(df['label'].unique())
print(f"Found {len(class_names)} classes: {class_names}")
label_to_id = {label: i for i, label in enumerate(class_names)}
df['numeric_label'] = df['label'].map(label_to_id)

In [ ]:
image_paths = df['full_path'].values
emotion_labels = df['numeric_label'].values
dataset = tf.data.Dataset.from_tensor_slices((image_paths, emotion_labels))

In [ ]:
def is_valid_data(path, label):
    is_valid = tf.py_function(
        lambda p: tf.io.gfile.exists(p.numpy().decode('utf-8')),
        [path],
        tf.bool
    )
    is_valid.set_shape([])
    return is_valid

In [ ]:
def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.io.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, IMG_SIZE)
    return image, label

In [ ]:
dataset = dataset.filter(is_valid_data)
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.repeat() # Added to prevent training from stopping after one epoch
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
print("\n✅ tf.data pipeline created successfully.")
print("Example batch from the dataset:")
for images, labels in dataset.take(1):
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [ ]:
print("--- Loading the base face recognition model ---")
saved_model = tf.keras.models.load_model('face_recognition_lfw_model.keras')

In [ ]:
print("--- Extracting the pre-trained base layer ---")
try:
    # Use the name we confirmed from the model summary
    base_model = saved_model.get_layer('efficientnetb2')
    base_model.trainable = False # Freeze the weights
    print("Base layer extracted and frozen successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure 'efficientnetb2' is the correct layer name from saved_model.summary().")

In [ ]:
NUM_EMOTION_CLASSES = len(class_names)

inputs = Input(shape=(260, 260, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(NUM_EMOTION_CLASSES, activation='softmax', name='emotion_output')(x)
emotion_model = Model(inputs=inputs, outputs=outputs)

print("\n--- New Emotion Model Summary ---")
emotion_model.summary()

In [ ]:
print("\n--- Starting Training on AffectNet ---")
emotion_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = emotion_model.fit(
    dataset,
    epochs=20,
    # steps_per_epoch is needed because the dataset repeats indefinitely
    steps_per_epoch=len(df) // BATCH_SIZE
)

In [ ]:
print("\n--- Saving the trained emotion model ---")
emotion_model.save('emotion_recognition_model.keras')
print("✅ Emotion model saved successfully!")

In [ ]:
import tensorflow as tf

# Load and re-save the Emotion Model
try:
    emotion_model = tf.keras.models.load_model('emotion_recognition_model.keras', compile=False)
    emotion_model.save('fixed_emotion_recognition_model.keras')
    print("Emotion model re-saved successfully!")
except Exception as e:
    print(f"Error re-saving emotion model: {e}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get one batch of images and labels from the dataset
image_batch, label_batch = next(iter(dataset))

# Make predictions on the batch
predictions = emotion_model.predict(image_batch)
predicted_indices = np.argmax(predictions, axis=1)

# The 'class_names' variable should still be available from your data prep script
# It's an array like ['anger', 'contempt', ...]
predicted_labels = [class_names[i] for i in predicted_indices]
actual_labels = [class_names[i] for i in label_batch.numpy()]

# Plot the results
plt.figure(figsize=(15, 15))
for i in range(min(9, BATCH_SIZE)):  # Display up to 9 images
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    plt.title(f"Actual: {actual_labels[i]}\nPredicted: {predicted_labels[i]}")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

# --- 1. Load All Your Trained Models ---
try:
    # NOTE: The emotion model might need the softmax if it was trained without one.
    # We will assume it does for this example.
    emotion_model = tf.keras.models.load_model('emotion_recognition_model.keras')
    emotion_class_names = ['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    # Load the face model you trained
    face_id_model = tf.keras.models.load_model('face_recognition_lfw_model.keras')

    # Get class names in the correct order
    print("Getting the official class names for the Face ID model...")
    temp_face_dataset = tf.keras.utils.image_dataset_from_directory(
        'lfw-subset-large/',
        image_size=(260, 260),
        batch_size=1
    )
    face_id_class_names = temp_face_dataset.class_names
    print(f"Found {len(face_id_class_names)} identities.")

    print("\nAll models loaded successfully.")
except Exception as e:
    print(f"Error loading a model or getting class names: {e}")

# --- 2. Prepare the Image for Both Models ---
image_path = '/content/beckham.jpeg'
img = tf.keras.utils.load_img(image_path, target_size=(260, 260))
img_array = tf.keras.utils.img_to_array(img)
img_array_batch = np.expand_dims(img_array, 0)

# --- 3. Run Predictions with Correct Labels ---
print("\n--- Running Full Analysis ---")

# Emotion Prediction
emotion_predictions = emotion_model.predict(img_array_batch)
emotion_score = tf.nn.softmax(emotion_predictions[0]) # Keep this if emotion model outputs logits
predicted_emotion = emotion_class_names[np.argmax(emotion_score)]
emotion_confidence = 100 * np.max(emotion_score)
print(f"Predicted Emotion: '{predicted_emotion}' ({emotion_confidence:.2f}%)")

# --- THIS IS THE FIX ---
# Face Identification Prediction
face_id_predictions = face_id_model.predict(img_array_batch)
# DO NOT apply softmax again. The model's output is already a probability distribution.
face_id_score = face_id_predictions[0]
predicted_person = face_id_class_names[np.argmax(face_id_score)]
face_id_confidence = 100 * np.max(face_id_score) # This will now show the true confidence
# --- END OF FIX ---

print(f"Closest Identity Match in LFW: '{predicted_person}' ({face_id_confidence:.2f}%)")

# --- 4. Display the Image ---
display(Image.open(image_path))